In [124]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score
import optuna

In [ ]:
data = pd.read_csv('train_sample_dropped.csv')

In [ ]:
y_data = data['flag']
X_data = data.drop('flag', axis=1)

In [131]:
X, X_test, y, y_test = train_test_split(X_data, y_data, test_size=0.1, random_state=42)

In [132]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [12]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 10, 60),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0, log=True),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0, log=True),
        "lambda_l1": trial.suggest_float("lambda1", 1e-3, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda2", 1e-3, 10.0, log=True),
        "verbosity": -1
    }
    all_f1 = []
    for i, (train_ind, val_ind) in enumerate(skf.split(X.values, y)):
        X_train, X_val = X.iloc[train_ind, :], X.iloc[val_ind, :]
        y_train, y_val = y.iloc[train_ind], y.iloc[val_ind]
        model = LGBMClassifier(**params, verbose=False)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        f1 = f1_score(y_pred, y_val)
        all_f1.append(f1) 
    
    return sum(all_f1) / len(all_f1)

# Запускаем оптимизацию
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30, show_progress_bar=True)

[I 2025-05-12 21:18:04,952] A new study created in memory with name: no-name-cb76d3b6-6f19-4b9e-a582-7077ec9776f9
  0%|          | 0/30 [00:00<?, ?it/s]c:\Users\zayda\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\zayda\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")
Best trial: 0. Best value: 0.707744:   3%|▎         | 1/30 [01:03<30:53, 63.91s/it]

[I 2025-05-12 21:19:08,855] Trial 0 finished with value: 0.7077443308397018 and parameters: {'n_estimators': 668, 'max_depth': 8, 'learning_rate': 0.02279443608487209, 'num_leaves': 53, 'bagging_fraction': 0.6265239507915653, 'feature_fraction': 0.9152684112019965, 'lambda1': 0.003243474576550798, 'lambda2': 0.6905350389594642}. Best is trial 0 with value: 0.7077443308397018.


Best trial: 0. Best value: 0.707744:   7%|▋         | 2/30 [01:13<14:48, 31.72s/it]

[I 2025-05-12 21:19:18,051] Trial 1 finished with value: 0.6885928370416233 and parameters: {'n_estimators': 69, 'max_depth': 4, 'learning_rate': 0.10064182315805366, 'num_leaves': 30, 'bagging_fraction': 0.6029166962365372, 'feature_fraction': 0.915489487271424, 'lambda1': 0.0016644375663454494, 'lambda2': 0.20973673358793712}. Best is trial 0 with value: 0.7077443308397018.


Best trial: 0. Best value: 0.707744:  10%|█         | 3/30 [02:06<18:42, 41.58s/it]

...

[I 2025-05-12 21:45:54,222] Trial 27 finished with value: 0.7095091421742492 and parameters: {'n_estimators': 889, 'max_depth': 10, 'learning_rate': 0.04705886835053195, 'num_leaves': 52, 'bagging_fraction': 0.9403396990774575, 'feature_fraction': 0.823886949639248, 'lambda1': 0.2831260875610823, 'lambda2': 4.664361664805743}. Best is trial 3 with value: 0.7102091788135013.


Best trial: 3. Best value: 0.710209:  97%|█████████▋| 29/30 [29:03<01:41, 101.43s/it]

[I 2025-05-12 21:47:08,845] Trial 28 finished with value: 0.7074523190549993 and parameters: {'n_estimators': 863, 'max_depth': 6, 'learning_rate': 0.07499418500801316, 'num_leaves': 36, 'bagging_fraction': 0.7374098729420856, 'feature_fraction': 0.6298298185002799, 'lambda1': 0.06885617353150639, 'lambda2': 0.4443328848292645}. Best is trial 3 with value: 0.7102091788135013.


Best trial: 3. Best value: 0.710209: 100%|██████████| 30/30 [30:54<00:00, 61.81s/it] 

[I 2025-05-12 21:48:59,206] Trial 29 finished with value: 0.7079334044266901 and parameters: {'n_estimators': 701, 'max_depth': 9, 'learning_rate': 0.02340782027185801, 'num_leaves': 52, 'bagging_fraction': 0.9532946700562546, 'feature_fraction': 0.6849752863015494, 'lambda1': 0.7256728703920281, 'lambda2': 1.309377745236475}. Best is trial 3 with value: 0.7102091788135013.


In [61]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [69]:
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10, log=True),
        "random_strength": trial.suggest_float("random_strength", 0.0, 10.0),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "verbose": 0,
    }
    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    return f1_score(y_pred, y_val)


# Запускаем оптимизацию
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2025-05-16 13:30:56,069] A new study created in memory with name: no-name-6bfe508a-84b2-43e2-bf24-f8a03f0eb318
[I 2025-05-16 13:31:08,315] Trial 0 finished with value: 0.6603482477407979 and parameters: {'iterations': 146, 'depth': 6, 'learning_rate': 0.02524110630916377, 'subsample': 0.6845452627352011, 'l2_leaf_reg': 0.2346421085337576, 'random_strength': 2.805363432343142, 'bagging_temperature': 0.019997649608886836, 'border_count': 112}. Best is trial 0 with value: 0.6603482477407979.
[I 2025-05-16 13:31:30,660] Trial 1 finished with value: 0.6972043010752688 and parameters: {'iterations': 320, 'depth': 6, 'learning_rate': 0.27494203663430433, 'subsample': 0.7306588490746503, 'l2_leaf_reg': 0.8487989443552472, 'random_strength': 6.526886259716379, 'bagging_temperature': 0.9382432232649237, 'border_count': 56}. Best is trial 1 with value: 0.6972043010752688.
[I 2025-05-16 13:32:10,388] Trial 2 finished with value: 0.6845901639344262 and parameters: {'iterations': 669, 'depth': 4,

In [135]:
study.best_params

{'n_estimators': 722,
 'max_depth': 10,
 'learning_rate': 0.044112234993877134,
 'subsample': 0.7967921767299626,
 'colsample_bytree': 0.9902645298264656,
 'lambda': 0.28898388933438157,
 'alpha': 0.88536586368353,
 'gamma': 3.2715697753064725e-06}